In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read
from astropy.nddata import StdDevUncertainty

In [2]:
# zHunter stuff
from zhunter import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb, get_vb_containing
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem
from zhunter.MainGraphicsWidget import MainGraphicsWidget
from zhunter.data_handler import DataHandler

In [3]:
# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [4]:
# astropalmerio stuff
import astropalmerio.spectra as sp
import astropalmerio.galaxies as gal
import astropalmerio.mc as mc


In [5]:
from astropalmerio.spectra import ergscm2AA
from astropalmerio.spectra import EmissionLine
from specutils import Spectrum1D

In [6]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [7]:
# Logging
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)
logging.getLogger("PyQt6.uic").setLevel(logging.WARNING)


In [8]:
## Colors
from zhunter.colors import COLORS
color_style = 'kraken9'
colors = COLORS[color_style]

## The data

In [9]:
# The fake data
N_data = 10000
N_spat = 40
wvlg = np.linspace(300, 900, N_data)
spat = np.linspace(-5, 5, N_spat)
flux = np.ones((wvlg.shape[0], spat.shape[0])) * np.exp(-(spat)**2/0.1)
# add noise
flux += np.random.normal(0, 0.1, size=(wvlg.shape[0], spat.shape[0]))
# add an emission line
flux[:,int(N_spat/2)] += sp.gaussian_fct(wvlg, mean=656.28, stddev=0.5, amplitude=3)
flux = flux.T
unc = flux*0.1
wvlg_tell = wvlg
wvlg_sky_bkg = wvlg
tellurics = 1 - np.exp(-(wvlg-600)**2/(30)**2)
sky_bkg = np.exp(-(wvlg-800)**2/(10)**2)

data = DataHandler()
data.load_2D(wvlg*u.nm, spat*u.arcsec, flux*ergscm2AA, unc*ergscm2AA)
data.load_1D(wvlg*u.nm, np.sum(flux[17:23], axis=0)*ergscm2AA, np.sqrt(unc[17:23]**2).sum(axis=0)*ergscm2AA)


2023-03-22 16:52:32,017 DEBUG [zhunter.misc] Flux and uncertainty need to be rescaled (exponent: -1), rescaling them.
2023-03-22 16:52:32,039 DEBUG [zhunter.misc] Flux and uncertainty need to be rescaled (exponent: -1), rescaling them.


## The display

In [10]:
from zhunter.LineFitGraphicsWidget import LineFitGraphicsWidget
from zhunter.fit_plot import LineFitPlot

In [11]:
lines = io.read_line_list('../../src/zhunter/data/lines/emission_lines.ecsv')


2023-03-22 16:52:32,060 DEBUG [zhunter.io] Reading line list ../../src/zhunter/data/lines/emission_lines.ecsv


In [12]:
fitPlot = LineFitPlot(lines=lines, z=0.0, data=data, mode='2D', colors=colors,)

2023-03-22 16:52:32,192 DEBUG [zhunter.fit_plot] Initialized with lines:
    name      awav              latex_str           
           Angstrom                                
---------- -------- -------------------------------
  Ly_alpha 1215.111                      Ly$\alpha$
  OII_3726 3726.032   [O\,\textsc{ii}]$\lambda3726$
  OII_3729 3728.815   [O\,\textsc{ii}]$\lambda3729$
NeIII_3869  3868.76 [Ne\,\textsc{iii}]$\lambda3869$
  HeI_3889 3888.647     He\,\textsc{i}$\lambda3889$
   H_delta 4101.742                       H$\delta$
   H_gamma 4340.471                       H$\gamma$
 OIII_4363  4363.21  [O\,\textsc{iii}]$\lambda4363$
    H_beta 4861.333                        H$\beta$
OIIIa_4959 4958.911  [O\,\textsc{iii}]$\lambda4959$
OIIIb_5007 5006.843  [O\,\textsc{iii}]$\lambda5007$
  HeI_5876 5875.624     He\,\textsc{i}$\lambda5876$
   OI_6300 6300.304    [O\,\textsc{i}]$\lambda6300$
  NII_6548  6548.05   [N\,\textsc{ii}]$\lambda6548$
   H_alpha 6562.819                       

In [13]:
fitPlot.show()

In [14]:
fitPlot.linefitLayout.ci.setBorder((100, 50, 100)) # this is to see where the Items' bounds are

In [15]:

fitPlot.visualize_spec()

2023-03-22 16:52:32,302 INFO [zhunter.fit_plot] Creating EmissionLine object with name H_alpha and redshift 0.0
2023-03-22 16:52:32,304 INFO [zhunter.MainGraphicsWidget] Setting up a new plot called 'H alpha' in '2D' mode, without Region Of Interest 
2023-03-22 16:52:32,519 INFO [zhunter.MainGraphicsWidget] Drawing 2D data
2023-03-22 16:52:32,552 DEBUG [zhunter.MainGraphicsWidget] Drawing 1D data


In [16]:
fitPlot.linefitLayout.draw_data()

2023-03-22 16:52:32,562 INFO [zhunter.MainGraphicsWidget] Drawing 2D data
2023-03-22 16:52:32,587 DEBUG [zhunter.MainGraphicsWidget] Drawing 1D data
2023-03-22 16:52:36,886 DEBUG [zhunter.MainGraphicsWidget] Key: C, Mouse position: [401,295]
2023-03-22 16:52:37,389 DEBUG [zhunter.MainGraphicsWidget] Key: C, Mouse position: [445,295]
2023-03-22 16:52:37,391 DEBUG [zhunter.LineFitGraphicsWidget] Updated fit bounds to (<Quantity 648.80119252 nm>, <Quantity 652.20713252 nm>)
2023-03-22 16:52:37,968 DEBUG [zhunter.MainGraphicsWidget] Key: C, Mouse position: [532,297]
2023-03-22 16:52:38,389 DEBUG [zhunter.MainGraphicsWidget] Key: C, Mouse position: [592,297]
2023-03-22 16:52:38,391 DEBUG [zhunter.LineFitGraphicsWidget] Updated fit bounds to (<Quantity 648.80119252 nm>, <Quantity 663.58606843 nm>)
2023-03-22 16:52:47,102 INFO [zhunter.fit_plot] Starting continuum fitting.
2023-03-22 16:52:47,104 DEBUG [astropalmerio.spectra.emission_lines] Attempting to extract the following bounds: (648.801

In [22]:
fitPlot.txe_fit_info.setText(fit_summary)

In [22]:
fitPlot.line.fit.keys()

dict_keys(['continuum', 'bounds', 'initial_guess', 'results', 'model', 'flux', 'residuals'])

In [38]:
fitPlot.line.fit['initial_guess']['amplitude']

<Quantity 33.11811521 0.1 ergscm2AA>

In [44]:
f"{fitPlot.line.fit['initial_guess']['amplitude'].to(u.Jy, equivalencies=u.spectral_density(fitPlot.line.fit['initial_guess']['mean'])):.3e}"


'4.757e+12 Jy'

In [26]:
fitPlot.line.properties

{'rest_awav': <Quantity 6562.819 Angstrom>,
 'z_guess': <Quantity 0.>,
 'obs_awav_guess': <Quantity 6562.819 Angstrom>,
 'detected': True,
 'obs_awav_fit': <Quantity 656.26492838 nm>,
 'FWHM_lam_fit': <Quantity 1.16210167 nm>,
 'FWHM_vel_fit': <Quantity 530.8668804 km / s>,
 'flux_fit': <Quantity 36.24626585 0.1 ergscm2AA nm>,
 'z_fit': <Quantity -2.58602597e-05>}

In [17]:
self = fitPlot.line


In [19]:
self.derive_properties_from_fit()

In [20]:
fit_summary = (
            "Fit summary\n"
            + "-------------\n"
            + f"Line name: {self.name}\n"
            + f"Fit bounds: ({self.fit['bounds'][0]:.3f}, {self.fit['bounds'][1]:.3f})\n"
            + f"Redshift: {self.properties['z_fit']:.5f}\n"

        )

In [21]:
fit_summary

'Fit summary\n-------------\nLine name: H_alpha\nFit bounds: (648.801 nm, 663.586 nm)\nRedshift: 0.00003\n'